In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
N = 64
spins = np.random.choice([-1, 1], (N,N))

def safe_spins(i,j):
    return spins[i,j] if 0 <= i < N and 0 <= j < N else 0

L = [(1,0), (-1,0), (0,1), (0,-1)]

def H(i,j,sigma):
    return sigma * sum(safe_spins(i+l[0],j+l[1]) for l in L)

def update(i,j,beta):
    w = np.array([np.exp(beta*H(i,j,1)), np.exp(beta*H(i,j,-1))])
    p = w / sum(w)
    return np.random.choice([1,-1], p=p)

def update_all(beta, alpha):
    mask = np.random.choice([0,1], p=[1-alpha,alpha], size=(N,N))
    new_spins = np.array([[update(i,j,beta) for j in range(N)] for i in range(N)])
    return new_spins * mask + spins * (1-mask)

bc = np.log(1+np.sqrt(2)) / 2

def beta_smooth(x):
    return bc * min(x+1, 3**3 * (x-1/3)**3 + 1)

In [3]:
for i in tqdm(range(1200)):
    beta = beta_smooth(i / 1200)

    for _ in range(25):
        spins = update_all(beta, 0.1)

    fig, (sp, b) = plt.subplots(1,2, figsize=(10,6))
    sp.axes.get_xaxis().set_visible(False)
    sp.axes.get_yaxis().set_visible(False)
    b.axes.get_xaxis().set_visible(False)
    b.axes.get_yaxis().set_visible(False)

    sp.matshow(spins,cmap='summer')

    b.set_aspect(5)
    b.axis('off')  # Turn off the axis
    b.set_xlim(-1,1)  # Set x limits to make the vertical line visible
    b.set_ylim(-0.01,2*bc + 0.01)

    b.axvline(x=0, color='black')  # Vertical line at x=0
    b.scatter(0, beta, zorder=3, color='red')  # Example point at y=10
    b.scatter(0, bc, zorder=2, color='black', marker='x')

    b.text(0.1, 0, 'low', verticalalignment='center')
    b.text(0.1, 2*bc, 'high', verticalalignment='center')
    b.text(0.1, bc, 'critical', verticalalignment='center')
    b.text(-0.15, beta, r'$\beta$', verticalalignment='center')


    plt.tight_layout()
    fig.subplots_adjust(wspace=-0.3)

    plt.savefig(f'images/{N}/{i:06d}.png', bbox_inches='tight')
    plt.close('all')

100%|██████████| 1200/1200 [24:01<00:00,  1.20s/it]
